pandas project

índice:
1. importación
2. exploración de los datos
3. toma de decisiones
4. data cleaning
5. análisis

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('GSAF5.csv', encoding = "ISO-8859-1")
data.head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2016.09.18.c,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.c-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.c,2016.09.18.c,5993,NaN,NaN
1,2016.09.18.b,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,Chucky Luciano,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.b-Luciano.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.b,2016.09.18.b,5992,NaN,NaN
2,2016.09.18.a,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.a-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.a,2016.09.18.a,5991,NaN,NaN
3,2016.09.17,17-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Thirteenth Beach,Surfing,Rory Angiolella,M,...,NaN,"The Age, 9/18/2016",2016.09.17-Angiolella.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.17,2016.09.17,5990,NaN,NaN
4,2016.09.15,16-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Bells Beach,Surfing,male,M,...,2 m shark,"The Age, 9/16/2016",2016.09.16-BellsBeach.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.16,2016.09.15,5989,NaN,NaN


In [3]:
data.shape

(5992, 24)

la importación daba un error:
UnicodeDecodeError: 'utf-8' codec can't decode byte 0x92 in position 1: invalid start byte
Busqué este error y di con la solución en stackoverflow, que era añadir 'encoding = "ISO-8859-1"' como argumento al llamar a pd.read_csv

In [4]:
data.columns = data.columns.str.strip().str.lower()
data.columns

Index(['case number', 'date', 'year', 'type', 'country', 'area', 'location',
       'activity', 'name', 'sex', 'age', 'injury', 'fatal (y/n)', 'time',
       'species', 'investigator or source', 'pdf', 'href formula', 'href',
       'case number.1', 'case number.2', 'original order', 'unnamed: 22',
       'unnamed: 23'],
      dtype='object')

renombro la columna case number con id, ya que es la que vamos a utilizar como campo clave

In [5]:
data = data.rename(columns = {'case number':'id'})

In [6]:
#vemos las columnas que tenemos y cuántos valores null tienen
null_cols = data.isnull().sum()
null_cols

id                           0
date                         0
year                         0
type                         0
country                     43
area                       402
location                   496
activity                   527
name                       200
sex                        567
age                       2681
injury                      27
fatal (y/n)                 19
time                      3213
species                   2934
investigator or source      15
pdf                          0
href formula                 1
href                         3
case number.1                0
case number.2                0
original order               0
unnamed: 22               5991
unnamed: 23               5990
dtype: int64

columnas Unnamed:22 y Unnamed:23 son todo nulls, por lo que habría que eliminarlas

columnas con información redundante, como las fechas. nos valdría con Year, no necesitamos Date

de la misma manera, con Country nos vale, no necesitamos Area ni Location

podríamos hacer estudio por países. en la gráfica mostraría solo los países en los que más ataques se producen

el campo Activity me parece interesante

voy a conservar el campo Fatal (Y/N)

estudiaremos solamente un periodo de varias décadas

estaría bien usar RegEx

nos quedamos con el Case Number, lo usaremos como id del evento

función apply

eliminar columnas con más de 2500 nulls

el campo Sex tiene un espacio. strip todos los campos

campo activity, agrupar fishing

surf skiing tiene dos nombres

In [7]:
data.groupby("type").count()[["id"]]

,id
type,
Boat,200
Boating,110
Invalid,519
Provoked,557
Sea Disaster,220
Unprovoked,4386


In [8]:
#vemos que tenemos boat y boating, así que unificaremos las dos

In [9]:
data.groupby("year").count()[["id"]].head()

,id
year,
0,124
5,1
77,1
500,1
1543,1


In [10]:
countries = data.groupby("country").count()[["id"]].sort_values("id", ascending=False)
display(countries.head())
countries.shape

,id
country,
USA,2116
AUSTRALIA,1279
SOUTH AFRICA,565
PAPUA NEW GUINEA,133
NEW ZEALAND,125


(203, 1)

In [11]:
data.groupby("activity").count()[["id"]].sort_values("id", ascending=False).head()

,id
activity,
Surfing,904
Swimming,819
Fishing,414
Spearfishing,321
Bathing,153


In [12]:
data.groupby("fatal (y/n)").count()[["id"]].sort_values("id", ascending=False)

,id
fatal (y/n),
N,4315
Y,1552
UNKNOWN,94
N,8
#VALUE!,1
F,1
N,1
n,1


ya sabemos los campos con los que nos vamos a quedar:

1. Case Number: id
2. Year: unidad temporal
3. Type: tipología del accidente
4. Country: país donde se dio el accidente
5. Activity: actividad que estaba haciendo la víctima
6. Sex: sexo de la víctima
7. Fatal (Y/N): fallecimiento o no de la víctima

pasos a seguir:
1. strip nombres de las columnas
2. corregir tipo de datos
3. strip columnas que sean strings
4. eliminar duplicados
5. eliminar las columnas con más de 2500 nulls
6. eliminar columnas que no vamos a utilizar
7. cambiar orden de las columnas
8. eliminar los años anteriores a 1920

He detectado que algunos nombres de las columnas tienen espacios al principio y/o al final, por lo que aplico strip(). Además, los pongo en minúsculas para que sea más cómodo escribir sus nombres.

eliminar duplicados

In [13]:
before = len(data)
data = data.drop_duplicates()
after = len(data)
print('Number of duplicate records dropped: ', str(before - after))

Number of duplicate records dropped:  0


eliminar columnas con más de 2500 nulls

In [14]:
print(data.shape)
drop_cols = list(null_cols[null_cols > 2500].index)
data = data.drop(drop_cols, axis=1)
print(data.shape)

(5992, 24)
(5992, 19)


vamos a coger el subset que queremos.
para ello, cogeremos solo las columnas que nos interesan

In [15]:
data = data[["id", "year", "type", "country", "activity", "fatal (y/n)"]]
data.head()
print(data.shape)

(5992, 6)


ahora vamos a quedarnos con los registros que contengan un año >= 1920, ya que vamos a analizar los últimos 100

In [16]:
data = data[data.year >= 1920]
data.shape

(5030, 6)

In [17]:
# comprobamos nulls
null_cols = data.isnull().sum()
null_cols

id               0
year             0
type             0
country         21
activity       395
fatal (y/n)      8
dtype: int64

In [18]:
# vemos que hay 21 registros sin país. vamos a eliminarlos

In [19]:
data = data[data.country.notnull()]

In [20]:
data.shape

(5009, 6)

vamos a cambiar el orden de las columnas, para ponerlas a nuestro gusto

In [22]:
column_order = ["id", "year", "country", "type", "activity", "fatal (y/n)" ]
data = data[column_order]
data.head()

,id,year,country,type,activity,fatal (y/n)
0,2016.09.18.c,2016,USA,Unprovoked,Surfing,N
1,2016.09.18.b,2016,USA,Unprovoked,Surfing,N
2,2016.09.18.a,2016,USA,Unprovoked,Surfing,N
3,2016.09.17,2016,AUSTRALIA,Unprovoked,Surfing,N
4,2016.09.15,2016,AUSTRALIA,Unprovoked,Surfing,N


In [23]:
# comprobamos si hay algún id duplicado. aquí tenemos que contar por una variable que no sea el id. cogemos year, ya que hemos visto que no tiene nulls
data.groupby("id").count()[["year"]].sort_values("year", ascending=False).head(14)

,year
id,
2005.04.06,2
1983.06.15,2
2006.09.02,2
2009.12.18,2
1980.07.00,2
2014.08.02,2
1962.06.11.b,2
2012.09.02.b,2
1966.12.26,2


In [ ]:
"""
vamos a asignar ids únicos a los que están duplicados
la siguiente función evalúa todos los ids y, en caso de que, a la hora de evaluar uno de ellos,
ya exista otro registro con ese mismo id, le pone una "a" al final, para diferenciarlo.
"""

In [25]:
def duplicates(id_column):
    new_id_column = []
    for i in range(len(id_column)):
        new_id = str(id_column[i])
        if new_id not in new_id_column:
            new_id_column.append(new_id)
        else:
            new_id_column.append(new_id + "a")
    return new_id_column

In [26]:
# aplicamos la función duplicates para la columna id
data["id"] = duplicates(list(data["id"]))

In [27]:
# comprobamos ahora que solo existe un registro para cada id
data.groupby("id").count()[["year"]].sort_values("year", ascending=False).head(14)

,year
id,
1920.00.00.a,1
2001.09.30.b,1
2002.01.01.a,1
2001.12.21,1
2001.11.23,1
2001.11.14,1
2001.11.07,1
2001.10.07,1
2001.10.02,1


In [28]:
# unificar type boat y boating
data = data.replace({'type': "Boating"}, "Boat")

In [29]:
# comprobación
data.groupby("type").count()[["id"]]

,id
type,
Boat,277
Invalid,409
Provoked,501
Sea Disaster,160
Unprovoked,3662


In [30]:
# vamos a aplicar strip() en la columna "fatal (y/n)" para unificar datos
data["fatal (y/n)"] = data["fatal (y/n)"].str.strip()
# además, vemos que hay un registro con F. entendemos que se refiere a fatal, así que lo cambiamos por "Y"
data = data.replace({"fatal (y/n)": "F"}, "Y")

In [31]:
# comprobación
data.groupby("fatal (y/n)").count()[["id"]].sort_values("id", ascending=False)

,id
fatal (y/n),
N,3895
Y,1040
UNKNOWN,67


In [33]:
null_cols = data.isnull().sum()
null_cols

id               0
year             0
country          0
type             0
activity       392
fatal (y/n)      7
dtype: int64